In [29]:
from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import os


In [44]:
import importlib
import bsm_utils
importlib.reload(bsm_utils)
from bsm_utils import bsm_calculator, csv_loader, sensitivity_test_new, greeks_calculator
from langchain.tools import Tool
import json
from dotenv import load_dotenv

In [ ]:
import os
os.environ["ANTHROPIC_API_KEY"] = "Your-API-Key-Here"


In [45]:
# bsm_tool = Tool(
#     name="BSM_Calculator",
#     func=black_scholes,
#     description="Calculates Black-Scholes-Merton option price. Inputs: option_type, S, K, T, r, sigma."
# )

# csv_reader_tool = Tool(
#     name="CSV_Reader",
#     func=read_csv_tool,
#     description="Reads a CSV file and returns the first few rows. Inputs: filepath (str), nrows (int, optional)."
# )
###Should we convert csv into json?###

llm = ChatAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-7-sonnet-latest", 
)

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a senior model developer in a bank. Your task is to draft sections of an ongoing performance assessment (OPA) report for a Black-Scholes option pricing model. Follow the OPA template and regulatory standards (SR11-7, ECB TRIM). Always use professional model development language, ensure logical flow."),
#     ("placeholder", "{chat_history}"),
#     ("human", "{input}"),
#     ("placeholder", "{agent_scratchpad}")
# ])



# Create the agent
tools = [bsm_calculator, csv_loader, sensitivity_test_new, greeks_calculator]

memory = MemorySaver()


In [46]:
for tool in tools:
    print(tool)
    print(tool.__dict__)

name='bsm_calculator' description="Calculates the Black-Scholes price for European call or put options.\nThis function implements the Black-Scholes formula to compute the theoretical price of a European option\n(either 'call' or 'put') given the current stock price, strike price, time to expiration, risk-free interest rate,\nand volatility of the underlying asset.\nArgs:\n    option_type (str): Type of the option, either 'call' or 'put'.\n    S (float): Current price of the underlying asset.\n    K (float): Strike price of the option.\n    T (float): Time to expiration in years.\n    r (float): Risk-free interest rate (annualized).\n    sigma (float): Volatility of the underlying asset (annualized standard deviation).\nReturns:\n    str: The calculated option price as a string.\nRaises:\n    ValueError: If option_type is not 'call' or 'put'.\n# AI Agent Explanation:\n# This function uses the Black-Scholes mathematical model to determine the fair price of a European option.\n# It comput

In [47]:
print(type(memory))
print(memory.__dict__)

<class 'langgraph.checkpoint.memory.InMemorySaver'>
{'serde': <langgraph.checkpoint.serde.jsonplus.JsonPlusSerializer object at 0x0000025CE0A2EA50>, 'storage': defaultdict(<function InMemorySaver.__init__.<locals>.<lambda> at 0x0000025D21D7A7A0>, {}), 'writes': defaultdict(<class 'dict'>, {}), 'blobs': defaultdict(None, {}), 'stack': <contextlib.ExitStack object at 0x0000025D21B09190>}


In [52]:
agent = create_react_agent(llm, tools)
#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent to invoke the greeks_calculator tool and print the result
def run_agents(print_table_after: bool = False):
    """Invoke the agent with an instruction to run the greeks_calculator tool and print the tool output.

    After the agent runs, optionally print the sensitivity table by calling the local helper
    `print_sensitivity_table_from_csv()` so you can see the tabular results in the notebook.
    """
    # Compose a human message that asks the agent to compute greeks for given inputs
    human_msg = ('human', "Read the first 2 rows from file in path '/Users/bobby/source/repos/model_doc_automation/dummy_options.csv' using the csv_loader as data. After the data is successfully read from the file, use the data to conduct sensitivity test. Parse the data in as input for sensitivity_test_new. The input should follow format below: option_type: str, S: float, K: float, T: float, r: float, sigma: float. Print a nice result table for the sensitivity test and provide a summary of the test for each row.")
    try:
        response = agent.invoke({
            "messages": [human_msg]
        })
    except Exception as e:
        print("Agent invocation failed:", e)
        return
    # The agent returns an object; try to print the last message content or the raw response
    try:
        print(response["messages"][-1].content)
    except Exception:
        print("Agent response:", response)


# Execute the agent example when this cell runs
run_agents()


## Sensitivity Test Results

### Row 1: Call Option Sensitivity Analysis
**Parameters:** S=100, K=105, T=1.0 year, r=5%, σ=20%

| Spot Change | Price    | Delta   | Gamma   | Vega     | Rho      | Theta    |
|-------------|----------|---------|---------|----------|----------|----------|
| -2.5%       | 6.73     | 0.4918  | 0.0205  | 38.89    | 41.22    | -5.95    |
| -2.0%       | 6.98     | 0.5020  | 0.0204  | 39.10    | 42.22    | -6.02    |
| -1.5%       | 7.23     | 0.5122  | 0.0202  | 39.28    | 43.22    | -6.09    |
| -1.0%       | 7.49     | 0.5222  | 0.0201  | 39.43    | 44.21    | -6.15    |
| -0.5%       | 7.75     | 0.5323  | 0.0200  | 39.56    | 45.21    | -6.22    |
| 0.0%        | 8.02     | 0.5422  | 0.0198  | 39.67    | 46.20    | -6.28    |
| 0.5%        | 8.29     | 0.5521  | 0.0197  | 39.75    | 47.19    | -6.33    |
| 1.0%        | 8.57     | 0.5619  | 0.0195  | 39.81    | 48.18    | -6.39    |
| 1.5%        | 8.86     | 0.5716  | 0.0193  | 39.84    | 49.16    | -6.

In [53]:
agent = create_react_agent(llm, tools)
#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run the agent to invoke the greeks_calculator tool and print the result
def run_agents(print_table_after: bool = False):
    """Invoke the agent with an instruction to run the greeks_calculator tool and print the tool output.

    This function prints the raw agent response robustly (avoids accidental truncation) and then
    optionally prints the sensitivity table using the local helper.
    """
    # Compose a human message that asks the agent to compute greeks for given inputs
    human_msg = ('human', "Read the first 2 rows from file in path '/Users/bobby/source/repos/model_doc_automation/dummy_options.csv' using the csv_loader as data. After the data is successfully read from the file, use the data to conduct sensitivity test. Parse the data in as input for sensitivity_test_new. The input should follow format below: option_type: str, S: float, K: float, T: float, r: float, sigma: float. Print a nice result table for the sensitivity test and provide a summary of the test for each row.")
    try:
        response = agent.invoke({
            "messages": [human_msg]
        })
    except Exception as e:
        print("Agent invocation failed:", e)
        return

    # Robustly inspect and print the response to avoid accidental truncation.
    import pprint, json
    print("Raw response type:", type(response))

    # Try to extract 'messages' if present, otherwise pretty-print the whole response
    messages = None
    try:
        messages = response.get("messages")
    except Exception:
        messages = None

    if not messages:
        # Fallback: print the entire response object
        pprint.pprint(response)
    else:
        for i, msg in enumerate(messages):
            # msg can be a dict-like or an object with a 'content' attribute
            content = None
            if isinstance(msg, dict):
                content = msg.get("content")
            else:
                content = getattr(msg, "content", None)

            # Print lengths and both repr and plain to avoid UI truncation surprises
            length = len(content) if isinstance(content, str) else None
            print(f"\n--- Message {i} (content length: {length}) ---")
            # repr helps reveal hidden characters/newlines that sometimes cause display issues
            print(repr(content))
            # plain print for readability
            print(content)

    # Optionally print the sensitivity table using the local helper
    if print_table_after:
        try:
            print("\n--- Sensitivity table (from sensitivity_test_new) ---")
            print_sensitivity_table_from_csv()
        except Exception as e:
            print("Failed to print sensitivity table:", e)

# Execute the agent example when this cell runs
run_agents()


Raw response type: <class 'dict'>

--- Message 0 (content length: 491) ---
"Read the first 2 rows from file in path '/Users/bobby/source/repos/model_doc_automation/dummy_options.csv' using the csv_loader as data. After the data is successfully read from the file, use the data to conduct sensitivity test. Parse the data in as input for sensitivity_test_new. The input should follow format below: option_type: str, S: float, K: float, T: float, r: float, sigma: float. Print a nice result table for the sensitivity test and provide a summary of the test for each row."
Read the first 2 rows from file in path '/Users/bobby/source/repos/model_doc_automation/dummy_options.csv' using the csv_loader as data. After the data is successfully read from the file, use the data to conduct sensitivity test. Parse the data in as input for sensitivity_test_new. The input should follow format below: option_type: str, S: float, K: float, T: float, r: float, sigma: float. Print a nice result table for the sens

In [18]:
import pandas as pd

pd.read_csv('/Users/bobby/source/repos/model_doc_automation/dummy_options.csv')

,date,S,K,T,r,sigma,option_type
0,2025-09-01,100,105,1.0,0.050,0.20,call
1,2025-09-02,102,106,0.9,0.045,0.19,put
2,2025-09-03,98,104,0.8,0.048,0.21,call
3,2025-09-04,101,107,0.7,0.047,0.18,call
4,2025-09-05,99,103,0.6,0.046,0.22,put
5,2025-09-06,103,108,0.5,0.049,0.20,put
6,2025-09-07,97,102,0.4,0.044,0.23,call
7,2025-09-08,100,106,0.3,0.050,0.19,call
8,2025-09-09,104,109,0.2,0.045,0.21,put
9,2025-09-10,96,101,0.1,0.048,0.20,put
